In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import pickle
import sys
import time
import torch

from scipy.integrate import solve_ivp

PROJECT_ROOT = os.path.abspath(
    os.path.join(os.getcwd(), os.pardir)
)
sys.path.append(PROJECT_ROOT)

import sample_points
from tcpinn import MLP, TcPINN
from plot import plot_solution_3d

np.random.seed(1)

In [2]:
if torch.cuda.is_available():
    device = torch.device("cuda")

else:
    device = torch.device("cpu")

In [3]:
class LorenzSystemInverse(TcPINN):
    """
    A tcPINN implementation for a Lorenz System inverse problem.
    """
    def __init__(
        self, layers, T, X_pinn=None, X_semigroup=None, X_smooth=None, 
        X_data=None, data=None, w_pinn=1., w_semigroup=1., w_smooth=1., w_data=1.
    ):
        """
        For inverse problems, one has to attach the trainable
        ODE parameters to the multilayer perceptron (MLP). The least confusing
        way to do this with the current implementation is to re-initialize 
        the MLP and the optimizer.
        """
        super().__init__(
            layers, T, X_pinn, X_semigroup, X_smooth, X_data, data,
            w_pinn, w_semigroup, w_smooth, w_data
        )
        self.is_inverse = True
        mlp = MLP(layers)
        self.mlp = self._init_ode_parameters(mlp).to(device)
        self.optimizer = torch.optim.LBFGS(
            self.mlp.parameters(), lr=1., max_iter=50000, max_eval=50000, 
            history_size=10, tolerance_grad=1e-5, tolerance_change=np.finfo(float).eps,
            line_search_fn="strong_wolfe"
        )
        self.scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
            self.optimizer, mode="min", min_lr=1e-4, verbose=True
        )     
    
    
    def _init_ode_parameters(self, mlp):

        mlp.sigma = torch.nn.Parameter(10 * torch.rand(1, requires_grad=True)[0])
        mlp.beta = torch.nn.Parameter(10 * torch.rand(1, requires_grad=True)[0])
        mlp.rho = torch.nn.Parameter(10 * torch.rand(1, requires_grad=True)[0])
        self.history["sigma"] = []
        self.history["beta"] = []
        self.history["rho"] = []

        return mlp
    
    
    def _loss_pinn(self):
        """
        The ODE-specific standard PINN loss.
        """
        y = self.net_y(self.t_pinn, self.y_pinn)
        deriv = self.net_derivative(self.t_pinn, self.y_pinn)
        
        loss1 = torch.mean(
            (deriv[0] - self.mlp.sigma * (y[:,1:2] - y[:,0:1])) ** 2
        )
        loss2 = torch.mean(
            (deriv[1] - y[:,0:1] * (self.mlp.rho - y[:,2:3]) + y[:,1:2]) ** 2
        )
        loss3 = torch.mean(
            (deriv[2] - y[:,0:1] * y[:,1:2] + self.mlp.beta * y[:,2:3]) ** 2
        )        
        loss = self.w_pinn * (loss1 + loss2 + loss3)

        return loss

### Setup Training Data

In [4]:
def rhs_lorenz_system(t, r, sigma, beta, rho):
    """
    Rhs of the Lorenz system
    """
    x, y, z = r
    
    fx = sigma * (y - x)
    fy = x * (rho - z) - y
    fz = x * y - beta * z
    
    return np.array([fx, fy, fz])


def get_solution(max_t, delta_t, init_val, sigma, beta, rho):

    times = np.linspace(0, max_t, int(max_t / delta_t) + 1)
    sol = solve_ivp(
        rhs_lorenz_system, [0, float(max_t)], init_val, t_eval=times,
        args=(sigma, beta, rho), rtol=1e-10, atol=1e-10
    )
    return sol.y.T


def sample_data(t_data, y_data, sigma, beta, rho):
    
    data = np.zeros((len(y_data), ode_dimension))

    for i, (t, init_val) in enumerate(zip(t_data, y_data)):

        data[i] = get_solution(
            max_t=t[0], delta_t=t[0], init_val=init_val,
            sigma=sigma, beta=beta, rho=rho
        )[-1]
    
    return data

In [5]:
# This example was used in:
# "DeepXDE: A deep learning library for solving differential equations", Lu et al.
# Note: With these parameters, all trajectories converge to either (4.9, 4.9, 9) or (-4.9, -4.9, 9)

sigma = 15.
beta = 8 / 3
rho = 10.

In [6]:
ode_dimension = 3
layers = [ode_dimension + 1] + 4 * [128] + [ode_dimension]

T = 1
max_y0 = 2

n_pinn = 10
t_pinn = np.random.uniform(0, T, (n_pinn, 1))
y_pinn = np.random.uniform(-max_y0, max_y0 , (n_pinn, ode_dimension))
X_pinn = np.hstack([t_pinn, y_pinn])

n_semigroup = 10
st_semigroup = sample_points.uniform_triangle_2d(n_semigroup, T)
y_semigroup = np.random.uniform(-max_y0, max_y0, (n_semigroup, ode_dimension))
X_semigroup = np.hstack([st_semigroup, y_semigroup])

n_smooth = 10
t_smooth = np.random.uniform(0, T, (n_smooth, 1))
y_smooth = np.random.uniform(-max_y0, max_y0, (n_smooth, ode_dimension))
X_smooth = np.hstack([t_smooth, y_smooth])

n_data = 10
t_data = np.random.uniform(0, T, (n_data, 1))
y_data = np.random.uniform(-max_y0, max_y0, (n_data, ode_dimension))
X_data = np.hstack([t_data, y_data])
data = sample_data(t_data, y_data, sigma, beta, rho)

In [7]:
model = LorenzSystemInverse(
    layers, T, X_pinn=X_pinn, X_semigroup=X_semigroup, X_smooth=X_smooth,
    X_data=X_data, data=data, w_data=10000
)

In [8]:
%%time
model.train()

CPU times: user 27.7 s, sys: 63.1 ms, total: 27.8 s
Wall time: 4.09 s


In [9]:
path = os.getcwd()

with open(f"{path}/model_lorenz_inverse.pkl", "wb") as handle:
    pickle.dump(model, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open(f"{path}/model_lorenz_inverse.pkl", "rb") as f:
    model = pickle.load(f)

In [ ]:
# plot inverse problem convergence
n_iterations = model.iter
sigma_true = np.full(n_iterations, sigma)
beta_true = np.full(n_iterations, beta)
rho_true = np.full(n_iterations, rho)

_, ax = plt.subplots(figsize=(10, 4))
ax.plot(np.arange(n_iterations), model.history["sigma"], color="green", label="sigma")
ax.plot(np.arange(n_iterations), model.history["beta"], color="blue", label="beta")
ax.plot(np.arange(n_iterations), model.history["rho"], color="orange", label="rho")
ax.plot(np.arange(n_iterations), sigma_true, color="black", linestyle="dashed")
ax.plot(np.arange(n_iterations), beta_true, color="black", linestyle="dashed")
ax.plot(np.arange(n_iterations), rho_true, color="black", linestyle="dashed")
ax.spines[["top", "right"]].set_visible(False)
ax.set_xlabel("Iterations")

plt.legend()
plt.show()

## Predict and Plot the Solution

In [11]:
# Note that max_t in training is 1
y0 = np.array([-1.25, -0.6, -0.4])# np.random.uniform(-max_y0, max_y0, 3)
max_t = 10
delta_t = 0.01

tc_solution = model.predict_tc(max_t, delta_t, y0)
true_solution = get_solution(max_t, delta_t, y0, sigma, beta, rho)

In [ ]:
ax = plot_solution_3d(true_solution, color="black", label="truth")
ax = plot_solution_3d(tc_solution, ax=ax, color="orange", linestyle="dashed", label="tcPINN")

plt.legend()
plt.show()